In [26]:
import tensorflow as tf
import json
from trainer import Trainer
from utils import map_word_to_vector
import numpy as np
import os

In [2]:
with open('102/checkpoints/model_configs.json', 'r') as inp:
    model_configs = json.load(inp)

In [3]:
with open('wiki_new/word2idx.json', 'r') as inp:
    new_word2idx = json.load(inp)
with open('wiki_new/char2idx.json', 'r') as inp:
    new_char2idx = json.load(inp)
with open('102/word2idx.json', 'r') as inp:
    old_word2idx = json.load(inp)
with open('102/char2idx.json', 'r') as inp:
    old_char2idx = json.load(inp)

In [4]:
VERSION = 102
params = dict(
    model_configs = model_configs,
    optimizer = tf.train.GradientDescentOptimizer,
    negative_samples = 8192,
    wdecay = 1.2e-6,
    alpha = 1e-6,
    beta = 1e-6,
    clip_norm = 0.3,
    bptt = 70,
    use_ema = True,
    save_freq = 1000,
    log_path = '{}/logs'.format(VERSION),
    train_summary_dir = '{}/train_summary/'.format(VERSION),
    test_summary_dir = '{}/test_summary/'.format(VERSION),
    checkpoint_dir = '{}/checkpoints/'.format(VERSION)
)

In [5]:
trainer = Trainer(**params)

In [6]:
trainer.build()

INFO:tensorflow:Restoring parameters from 102/checkpoints/train/model.cpkt-557539


Restoring parameters from 102/checkpoints/train/model.cpkt-557539


In [7]:
model_configs['vocab_size'] = len(new_word2idx) + 1
model_configs['char_vocab_size'] = len(new_char2idx) + 1

In [8]:
tf.global_variables()

[<tf.Variable 'LanguageModel/decode_W:0' shape=(512, 153360) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/decode_b:0' shape=(153360,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/embedding_weight:0' shape=(1005, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d/kernel:0' shape=(1, 16, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_1/kernel:0' shape=(2, 16, 16) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_1/bias:0' shape=(16,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_2/kernel:0' shape=(3, 16, 32) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_2/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_3/kernel:0' shape=(4, 16, 64) dtype=float32_ref>,
 <tf.Variable 'LanguageModel/word_embedding/conv1d_3/bias:0' s

In [9]:
with tf.variable_scope('LanguageModel', reuse=True):
    W_var = tf.get_variable('decode_W')
    b_var = tf.get_variable('decode_b')

In [10]:
W_val = trainer.session.run(W_var)
b_val = trainer.session.run(b_var)

In [16]:
new_W_val = map_word_to_vector(new_word2idx, old_word2idx, W_val.T).T
new_b_val = np.squeeze(map_word_to_vector(new_word2idx, old_word2idx, np.expand_dims(b_val, axis=-1)), axis=-1)

In [20]:
assign_W = tf.assign(W_var, new_W_val, validate_shape=False)
assign_b = tf.assign(b_var, new_b_val, validate_shape=False)

In [21]:
trainer.session.run([assign_W, assign_b])

[array([[-0.03200835,  0.00996768,  0.02794232, ...,  0.01263673,
          0.00457643,  0.01547284],
        [ 0.00662694, -0.04005435, -0.08283629, ..., -0.00105607,
          0.00235879,  0.00252642],
        [-0.01038062, -0.00208524,  0.1434482 , ..., -0.00717628,
         -0.008939  , -0.00799257],
        ...,
        [-0.01310354, -0.07520254, -0.09203153, ...,  0.00551493,
          0.00793004,  0.02744709],
        [ 0.01507463,  0.03159498, -0.04392492, ..., -0.00180185,
         -0.00248681, -0.01216666],
        [-0.00925479,  0.02840179,  0.00848963, ...,  0.00265031,
          0.01340717, -0.00341817]], dtype=float32),
 array([-2.7989112e-02,  7.9498434e+00,  3.5838830e+00, ...,
        -7.6987548e-03, -1.0311298e-02, -6.9140429e-03], dtype=float32)]

In [24]:
trainer.checkpoint_dir = '102_new/checkpoints/'
trainer.model_configs = model_configs
trainer.save_configs()

In [27]:
trainer.train_saver.save(trainer.session, os.path.join(trainer.checkpoint_dir, 'train', 'model.cpkt'), global_step=trainer.session.run(trainer.global_step))

'102_new/checkpoints/train/model.cpkt-557539'

In [29]:
trainer.test_saver.save(trainer.session, os.path.join(trainer.checkpoint_dir, 'test', 'model.cpkt'), global_step=trainer.session.run(trainer.global_step))

'102_new/checkpoints/test/model.cpkt-557539'